# Text-Fabric

This is testing ground for algorithms to be used in Text-Fabric.

## Range manipulation

### Convert iterable to ranges
Convert an iterable of numbers to a comma separated optimal list of ranges
optimal means:
1. the ranges are sorted
2. adjacent ranges do not share a boundary, i.e. there is a real gap
   between adjacent ranges
**NB:** Unlike in Python, ranges include their boundary values.

The parameter `nlist` can be any iterable, arrays, lists, 
dictionary keys, sets but the items must be numbers, not strings.
The iterable will be sorted and stripped of duplicates.

We also define the converse: from a list of number ranges to an iterable, in this case: a list. The `ranges` parameter is a list of tuples `start` and `end`, which are the boundaries of a range. The boundaries are inclusive. Instead of a tuple, a single integer may be provided.
If `end` is bigger than `start`, the two will be swapped around.
The result will be a sorted list without duplicates.

In [1]:
import collections

In [2]:
def convert_to_ranges(nlist):
    ranges = []
    curstart = None
    curend = None
    for n in sorted(set(nlist)):
        if curstart == None:
            curstart = n
            curend = n
        elif n == curend + 1:
            curend = n
        else:
            ranges.append((curstart, curend))
            curstart = n
            curend = n
    if curstart != None:
        ranges.append((curstart, curend))
    return ranges

In [3]:
def ranges_from_text(ranges_text):
    result = []
    for r_str in ranges_text.split(','):
        bounds = r_str.split('-')
        if len(bounds) == 1:
            result.append(int(bounds[0]))
        else:
            result.append((int(bounds[0]), int(bounds[1])))
    return result

def ranges_to_list(ranges):
    covered = set()
    for r in ranges:
        if type(r) is tuple:
            (start, end) = r
            if start > end:
                (end, start) = r
            for i in range(start, end+1):
                covered.add(i)
        else:
            covered.add(r)
    return sorted(covered)

In [4]:
def ranges_rep(ranges):
    return ','.join(
        str(r) if type(r) is int else\
        str(r[0]) if len(r) == 1 or r[0]==r[1] else\
        '{}-{}'.format(*r)\
        for r in ranges
    )   

In [5]:
tests_r2l = dict(
    a_empty = [],
    b_simple = [(5,10)],
    c_swapped = [(10,5)],
    d_single = [1,2,3,4,5],
    e_mixed = [1,2, (10,15), 16],
    f_duplicate = [1, (1,5), (3,7), 7],
    g_order = [7, (1,5), (7,3), 1],
)

In [6]:
for (t, data) in sorted(tests_r2l.items()):
    print('[{:<12}] {}'.format(t, ranges_to_list(data)))

[a_empty     ] []
[b_simple    ] [5, 6, 7, 8, 9, 10]
[c_swapped   ] [5, 6, 7, 8, 9, 10]
[d_single    ] [1, 2, 3, 4, 5]
[e_mixed     ] [1, 2, 10, 11, 12, 13, 14, 15, 16]
[f_duplicate ] [1, 2, 3, 4, 5, 6, 7]
[g_order     ] [1, 2, 3, 4, 5, 6, 7]


In [7]:
tests_l2r = dict(
    a_empty = [],
    b_consec = range(5,10),
    c_mixed = ranges_to_list([(1,4), (5,8), (10,12), (100, 110)]),
    d_mixed_up = [1,2, 5,6,7, 2,3, 3,4, 12,11,10],
)

In [8]:
for (t, data) in sorted(tests_l2r.items()):
    print('[{:<12}] {}'.format(t, convert_to_ranges(data)))

[a_empty     ] []
[b_consec    ] [(5, 9)]
[c_mixed     ] [(1, 8), (10, 12), (100, 110)]
[d_mixed_up  ] [(1, 7), (10, 12)]


# Exporting otype
We want to export the `otype` feature in compact form, like this:

    0-425000 word
    500-700000 phrase

and so on for each object type.

We do the exercise for the brandnew data version ETCBC4c.

In [9]:
from laf.fabric import LafFabric
from etcbc.preprocess import prep
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.8.3
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: https://shebanq.ancient-data.org/static/docs/featuredoc/texts/welcome.html



In [10]:
fabric.load('etcbc4c', '--', 'TF', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        otype monads g_word_utf8 trailer_utf8
        sp
    ''','''
        mother
        functional_parent
    '''),
    "prepare": prep(select={'L'}),
    "primary": False,
}, verbose='DETAIL')
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s DETAIL: COMPILING m: etcbc4c: UP TO DATE
  0.00s USING main: etcbc4c DATA COMPILED AT: 2016-11-09T19-16-37
  0.01s DETAIL: load main: G.node_anchor_min
  0.10s DETAIL: load main: G.node_anchor_max
  0.14s DETAIL: load main: G.node_sort
  0.19s DETAIL: load main: G.node_sort_inv
  0.56s DETAIL: load main: G.edges_from
  0.62s DETAIL: load main: G.edges_to
  0.68s DETAIL: load main: F.etcbc4_db_monads [node] 
  1.34s DETAIL: load main: F.etcbc4_db_otype [node] 
  1.96s DETAIL: load main: F.etcbc4_ft_g_word_utf8 [node] 
  2.24s DETAIL: load main: F.etcbc4_ft_sp [node] 
  2.40s DETAIL: load main: F.etcbc4_ft_trailer_utf8 [node] 
  2.51s DETAIL: load main: F.etcbc4_ft_functional_parent [e] 
  2.75s DETAIL: load main: F.etcbc4_ft_mother [e] 
  2.81s DETAIL: load main: C.etcbc4_ft_functional_parent -> 
  3.52s DETAIL: load main: C.etcbc4_ft_mother -> 
  3.66s DETAIL: load main: C.etcbc4_ft_functional_parent <- 
  4.19s DETAIL: load main: C.etcbc4_

## otype

In [11]:
otypes = '''
    word
    subphrase
    phrase_atom
    phrase
    clause_atom
    clause
    sentence_atom
    sentence
    half_verse
    verse
    chapter
    book
'''.strip().split()

In [12]:
ranges = {}
for otype in otypes:
    these_ranges = convert_to_ranges(F.otype.s(otype))
    inf('{:<15} {:>5} ranges'.format(
        otype, len(these_ranges),
    ), withtime=True)
    ranges[otype] = these_ranges
for (otype, ((start, end),)) in sorted(ranges.items(), key=lambda x: x[1][0][0]):
    print('{:<15} from {:>7} to {:>7}'.format(otype, start, end))

    27s word                1 ranges
    28s subphrase           1 ranges
    29s phrase_atom         1 ranges
    30s phrase              1 ranges
    31s clause_atom         1 ranges
    32s clause              1 ranges
    33s sentence_atom       1 ranges
    34s sentence            1 ranges
    35s half_verse          1 ranges
    36s verse               1 ranges
    38s chapter             1 ranges
    39s book                1 ranges
word            from       0 to  426580
clause          from  426581 to  514580
clause_atom     from  514581 to  605142
phrase          from  605143 to  858316
phrase_atom     from  858317 to 1125831
sentence        from 1125832 to 1189401
sentence_atom   from 1189402 to 1253740
subphrase       from 1253741 to 1367532
book            from 1367533 to 1367571
chapter         from 1367572 to 1368500
half_verse      from 1368501 to 1413680
verse           from 1413681 to 1436893


In a slightly different format:

In [13]:
of = outfile('otype.tf')
for (otype, ((start, end),)) in sorted(ranges.items(), key=lambda x: x[1][0][0]):
    print('{}-{}\t{}'.format(start, end, otype))
    of.write('{}-{}\t{}\n'.format(start, end, otype))
of.close()

0-426580	word
426581-514580	clause
514581-605142	clause_atom
605143-858316	phrase
858317-1125831	phrase_atom
1125832-1189401	sentence
1189402-1253740	sentence_atom
1253741-1367532	subphrase
1367533-1367571	book
1367572-1368500	chapter
1368501-1413680	half_verse
1413681-1436893	verse


### Map monad numbers
In TF we make sure that the monads go from 0-maxmonad consecutively. So we have to map the original monad numbers
to the node numbers of the words in TF.

In [15]:
tfFromMonad = {}
for w in F.otype.s('word'):
    m = int(F.monads.v(w))
    tfFromMonad[m] = w
of = outfile('tfFromMonad.csv')
of.write('monad\tTF\n')
for x in sorted(tfFromMonad.items()):
    of.write('{}\t{}\n'.format(*x))
of.close()

In [20]:
def tfFromMonadList(mList): return [tfFromMonad[m] for m in mList]

## monads

Here is code to write edge information in compact text files.
We select a domain with a very limited amount of nodes, and collect all edges between them.

In [17]:
first_object = {}
for otype in otypes:
    first_object[otype] = ranges[otype][0][0]
    print('The first {} object is {}'.format(otype, first_object[otype]))

The first word object is 0
The first subphrase object is 1253741
The first phrase_atom object is 858317
The first phrase object is 605143
The first clause_atom object is 514581
The first clause object is 426581
The first sentence_atom object is 1189402
The first sentence object is 1125832
The first half_verse object is 1368501
The first verse object is 1413681
The first chapter object is 1367572
The first book object is 1367533


In [19]:
for otype in reversed(otypes[1:]):
    for n in range(first_object[otype], first_object[otype]+2):
        print('{}\t{}\t'.format(n, F.monads.v(n)))

1367533	1-28762	
1367534	28763-52510	
1367572	1-673	
1367573	674-1167	
1413681	1-11	
1413682	12-31	
1368501	1-4	
1368502	5-11	
1125832	1-11	
1125833	12-18	
1189402	1-11	
1189403	12-18	
426581	1-11	
426582	12-18	
514581	1-11	
514582	12-18	
605143	1-2	
605144	3	
858317	1-2	
858318	3	
1253741	5-7	
1253742	9-11	


## A few features

First node features.

In [16]:
nfeats = '''
    g_word_utf8
    trailer_utf8
    sp
'''.strip().split()

efeats = '''
    mother
    functional_parent
'''.strip().split()

In [17]:
def escape_ft(s):
    return s.replace('\\', '\\\\').replace('\t', '\\t').replace('\n', '\\n').replace(' ', '_')

for feat in nfeats:
    print('\n[{}]\n'.format(feat))
    for n in range(20):
        print(escape_ft(F.item[feat].v(n)))


[g_word_utf8]

בְּ
רֵאשִׁ֖ית
בָּרָ֣א
אֱלֹהִ֑ים
אֵ֥ת
הַ
שָּׁמַ֖יִם
וְ
אֵ֥ת
הָ
אָֽרֶץ
וְ
הָ
אָ֗רֶץ
הָיְתָ֥ה
תֹ֨הוּ֙
וָ
בֹ֔הוּ
וְ
חֹ֖שֶׁךְ

[trailer_utf8]


_
_
_
_

_

_

׃_


_
_
_

_

_

[sp]

prep
subs
verb
subs
prep
art
subs
conj
prep
art
subs
conj
art
subs
verb
subs
conj
subs
conj
subs


Then an edge feature: `mother`. We list all the edges involving the first eleven words, and the objects that surround them.
We follow all edges and inverted edges until we reach end points.
Exclude chains of `clause_atom`s.

In [18]:
words = set(range(11))
domain = {}

for feat in efeats:
    print('Analyzing edge feature "{}"'.format(feat))
    unchecked_nodes = set()
    for w in words:
        unchecked_nodes |= {w} | {L.u(otype, w) for otype in otypes[1:] if L.u(otype, w) != None}        
    print('Starting with {} nodes'.format(len(unchecked_nodes)))
    checked_nodes = set()
    max_iter = 1000
    i = 0
    while len(unchecked_nodes):
        i +=1
        if i >= max_iter:
            print('Iteration {:>3}\n!Max iterations reached!\n{:>3} unchecked nodes\n{:>3} checked nodes'.format(
                i,
                len(unchecked_nodes),
                len(checked_nodes),
            ))
            break
        n = sorted(unchecked_nodes)[0]
        to_edges = {m for m in C.mother.v(n) if F.otype.v(m) != F.otype.v(n)}
        from_edges = {m for m in Ci.mother.v(n) if F.otype.v(m) != F.otype.v(n)}
        checked_nodes.add(n)
        unchecked_nodes.discard(n)
        unchecked_nodes |= (to_edges | from_edges) - checked_nodes
    print('Iteration {:>3}\n{:>3} unchecked nodes\n{:>3} checked nodes'.format(
        i,
        len(unchecked_nodes),
        len(checked_nodes),
    ))
    # what have we got?
    for n in sorted(checked_nodes):
        print('{}\t{}'.format(n, F.otype.v(n)))
    domain[feat] = checked_nodes
    print('{} nodes'.format(len(checked_nodes)))

Analyzing edge feature "mother"
Starting with 30 nodes
Iteration  30
  0 unchecked nodes
 30 checked nodes
0	word
1	word
2	word
3	word
4	word
5	word
6	word
7	word
8	word
9	word
10	word
426581	clause
514581	clause_atom
605143	phrase
605144	phrase
605145	phrase
605146	phrase
858317	phrase_atom
858318	phrase_atom
858319	phrase_atom
858320	phrase_atom
1125832	sentence
1189402	sentence_atom
1253741	subphrase
1253742	subphrase
1367533	book
1367572	chapter
1368501	half_verse
1368502	half_verse
1413681	verse
30 nodes
Analyzing edge feature "functional_parent"
Starting with 30 nodes
Iteration  30
  0 unchecked nodes
 30 checked nodes
0	word
1	word
2	word
3	word
4	word
5	word
6	word
7	word
8	word
9	word
10	word
426581	clause
514581	clause_atom
605143	phrase
605144	phrase
605145	phrase
605146	phrase
858317	phrase_atom
858318	phrase_atom
858319	phrase_atom
858320	phrase_atom
1125832	sentence
1189402	sentence_atom
1253741	subphrase
1253742	subphrase
1367533	book
1367572	chapter
1368501	half_verse
1

In [19]:
for feat in efeats:
    print('Showing edges for "{}"'.format(feat))
    condensed_end_points = collections.defaultdict(set)
    for n in domain[feat]:
        edges = set(C.item[feat].v(n))
        if len(edges):
            condensed_end_points[ranges_rep(convert_to_ranges(edges))].add(n)
    condensed = collections.defaultdict(set)
    for (end_points_rep, start_points) in condensed_end_points.items():
        condensed[ranges_rep(convert_to_ranges(start_points))].add(end_points_rep)
    for (start_points_rep, end_points_rep) in sorted(condensed.items()):
        print('{}\t{}\t'.format(start_points_rep, ','.join(sorted(end_points_rep))))
        

Showing edges for "mother"
1253742	1253741	
Showing edges for "functional_parent"
0-1,858317	605143	
2,858318	605144	
3,858319	605145	
4-10,858320	605146	
426581,1189402	1125832	
514581,605143-605146	426581	


## Export the monads edge feature compactly

We have already a mapping from nodes to monad set: the monads feature.
We also want to collapse lines that assign the same monadset to several nodes.
And, to be sure, we normalize all ranges we encounter, not assuming that they are already normalized.

**NB** For the monads feature this is not the best way. We gain more space efficiency by listing for each node its
associated monads on one line, because then we can leave out the node number.
We implement that in the next cell.

In [21]:
normalized_monads = collections.defaultdict(set)
chunksize = 100000
i = 0
j = 0
inf('Normalizing the monads feature')
for n in NN():
    if F.otype.v(n) == 'word': continue
    i += 1
    j += 1
    if j == chunksize:
        j = 0
        inf('{:>7} nodes'.format(i))
    normalized_monads[
        ranges_rep(convert_to_ranges(tfFromMonadList(ranges_to_list(ranges_from_text(F.monads.v(n))))))
    ].add(n)
inf('{:>7} nodes map to {:>7} monad sets'.format(i, len(normalized_monads)))

18m 04s Normalizing the monads feature
18m 05s  100000 nodes
18m 06s  200000 nodes
18m 07s  300000 nodes
18m 09s  400000 nodes
18m 11s  500000 nodes
18m 12s  600000 nodes
18m 13s  700000 nodes
18m 14s  800000 nodes
18m 15s  900000 nodes
18m 17s 1000000 nodes
18m 17s 1010313 nodes map to  527633 monad sets


Now condense the list.

In [22]:
inf('Condensing')
of = outfile('monads_plain.tf')
condensed_monads = collections.defaultdict(set)
for (monads_rep, node_set) in normalized_monads.items():
    condensed_monads[ranges_rep(convert_to_ranges(node_set))].add(monads_rep)
for (node_set_rep, monad_set_rep) in sorted(condensed_monads.items()):
    of.write('{}\t{}\t\n'.format(node_set_rep, ','.join(sorted(monad_set_rep))))
of.close()
inf('Done')

18m 27s Condensing
18m 33s Done


Here is the other way of representing the monads edge feature

In [23]:
max(ranges[otype][-1][1] for otype in otypes[1:])

1436893

In [24]:
inf('Writing the monads feature')
of = outfile('monads.tf')
i = 0
start_node = ranges[otypes[0]][-1][1] + 1 # one more than the last word
end_node = max(ranges[otype][-1][1] for otype in otypes[1:])

of.write('{}\t'.format(start_node))
for n in range(start_node, end_node+1):
    i+=1
    of.write('{}\t\n'.format(
        ranges_rep(convert_to_ranges(tfFromMonadList(ranges_to_list(ranges_from_text(F.monads.v(n))))))
    ))
of.close()
inf('{:>7} nodes written'.format(i))

20m 39s Writing the monads feature
20m 49s 1010313 nodes written


In [31]:
inf('Loading monads')
fl = infile('monads.tf')
monads_intern = collections.defaultdict(set)
for line in fl:
    (nodes, monads, label) = line.rstrip('\n').split('\t')
    node_list = ranges_to_list(ranges_from_text(nodes))
    monads_list = ranges_to_list(ranges_from_text(monads))
    for n in node_list:
        monads_intern[n] |= set(monads_list)
fl.close()
inf('Monads loaded monad specs for {} nodes'.format(len(monads_intern)))

55m 24s Loading monads
55m 31s Monads loaded monad specs for 1010313 nodes


## Reading the monads file
How fast can the monads file be read and stored as a list valued dictionary of nodes?

# Exporting a TF set

We will export the ETCBC4c as a Text-Fabric dataset.

That means: a *backbone* consisting of the `otype` feature and the `extent` edge.

On top of that: the feature files. At first, we just pick two: `g_word_utf8` and `trailer_utf8`.
Together they form exactly the plain text of the Hebrew Bible in ETCBC4b.

The `otype` feature maps every node id onto it object type (otype).

The `extent` edge feature contains unlabeled edges between each object and every *monad* belonging to it.

In [ ]:
def tf_otype():